In [1]:
import parselmouth as pm
import numpy as np
from fasttrackpy.tracks import OneTrack, CandidateTracks
from fasttrackpy.processors.smoothers import Smoother
import plotly.express as px
import pandas as pd
import polars as pl
import time
from IPython.display import Image
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Read in the sound file with parselmouth

In [2]:
sound = pm.Sound("assets/ay.wav")

In [3]:
x = OneTrack(sound = sound, maximum_formant=4000)

In [4]:
start = time.time()
foo = x.formant_obj
end = time.time()
end-start

0.010396957397460938

In [5]:
start = time.time()
foo = x.formant_obj
end = time.time()
end-start

0.01129913330078125